In [240]:
#workhorses
import numpy as np
import pandas as pd


#plotting things

#%matplotlib qt5 -- I don't know what this is
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from cycler import cycler

In [241]:
#All of Anandh's customized seaborn/matplotlib settings

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

#%config InlineBackend.figure_f.ormats=['svg']

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#mpl.rc('text', usetex=False)
#mpl.rc('text.latex', preamble=r'\usepackage{helvet}
#\renewcommand\familydefault{\sfdefault}\usepackage{sansmath}\sansmath')

    #If you want to use a different font
# mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 
#                  'serif': ['Helvetica']})

tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('figure', figsize=(9,8))

## Get the data

In [242]:
data_dict = pd.read_excel('../Local Data/20181004 A=B community screen 1 no sample.xlsx', sheet_name=None)

In [243]:
data_dict.keys()

odict_keys(['OD700', 'BFP', 'YFP', 'IDs', 'Exp'])

## Munge the data

### Functions

In [244]:
def get_sec (datetimeTime_obj):
    
    return (datetimeTime_obj.hour * 60 + datetimeTime_obj.minute) * 60 + datetimeTime_obj.second

In [245]:
def replace_time (data_sheet):
    
    df = data_sheet.copy()
    
    df['Time'] = df['Time'].apply(get_sec).divide(3600)
    
    return df

In [246]:
def replace_temp (data_sheet):
    
    df = data_sheet.copy()
    
    c = ['Temp C' if '°' in x else x for x in df.columns]
    
    df.columns = c
    
    return df

In [247]:
def add_id_info (data_sheet_melted, df_id):
    
    #make a copy of the dataframe so you can return the new one and set whatever name you want
    df = data_sheet_melted.copy()

    #initialize the columns to be used to store ID info
    all_id_columns = [col for col in df_id.columns if col not in 'well']

    for c in all_id_columns:
        df[c] = np.nan

    #get the list of all the wells you want to annotate
    wells = np.unique(df_id['well'])

    #loop over these wells and add the information to the initialized columns
    for w in wells:
        #get the ID information for this well
        append_this = df_id.loc[df_id['well'] == w, :]

        #loop over each column containing a unique piece of ID info
        for c in all_id_columns:

            #set the value for that column for that well
            df.loc[df['well'] == w, c] = append_this[c].values
                                                                #have to use .values otherwise index carries along and only
                                                                #sets the value for the data table index equal to the
                                                                #id table index

    return df

## Do the munging

In [248]:
keys_to_munge = ['OD700', 'BFP', 'YFP']

#replace time and temp by overwriting the original data
for key in keys_to_munge:
    
    data_dict[key] = replace_time(data_dict[key])
    data_dict[key] = replace_temp(data_dict[key])

### Assign well IDs

In [219]:
#get the well identifying information
ids = data_dict['IDs']

#get the data you want
od_raw = data_dict['OD700']
bfp_raw = data_dict['BFP']
yfp_raw = data_dict['YFP']

In [220]:
#independently melt the data so you can check and control the ID vars and value vars since that might differ per expt
od = pd.melt(od_raw, id_vars=['Time', 'Temp C'], var_name=['well'])

bfp = pd.melt(bfp_raw, id_vars=['Time', 'Temp C'], var_name=['well'])

yfp = pd.melt(yfp_raw, id_vars=['Time', 'Temp C'], var_name=['well'])

In [232]:
od_final = add_id_info(od, ids)

bfp_final = add_id_info(bfp, ids)

yfp_final = add_id_info(yfp, ids)

## Write the data to a new file